<a href="https://colab.research.google.com/github/ayemish/sharkstack/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip uninstall -y pinecone-client pinecone
!pip install pinecone langchain langchain-pinecone langchain-community langchain-text-splitters sentence-transformers pypdf


In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore


In [ ]:
os.environ["PINECONE_API_KEY"] = "pcsk_6TDdRa_Eu5mZiJM1TX1cjTstQ9Ku8NxvYEFjXaFzfVkt63tgud7oJkqx2aHpgKJcKjGQSp"


In [ ]:
loader = PyPDFLoader("book.pdf")
documents = loader.load()
print(f"Total pages loaded: {len(documents)}")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n", " ", ""]
)
chunks = text_splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)}")


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "book-rag-chatbot"

# create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # embedding size for MiniLM-L6-v2
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# wait a bit before proceeding (index takes some seconds to initialize)


In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)


In [ ]:
from transformers import pipeline

# load a small QA model
qa_model = pipeline("text2text-generation", model="google/flan-t5-small")

def rag_chatbot(query):
    # search in Pinecone
    results = vectorstore.similarity_search(query, k=3)
    context = " ".join([doc.page_content for doc in results])

    # instruction: only answer from context
    prompt = f"""Answer the question using ONLY the following context:

    {context}

    Question: {query}

    If the answer is not in the context, say: "I could not find it."
    """

    output = qa_model(prompt, max_length=256, do_sample=False)
    return output[0]['generated_text']

# Example
print(rag_chatbot("What is the main theme of chapter 5?"))


In [ ]:
rag_chatbot("Summarize chapter 2.")
rag_chatbot("Who are the main characters?")
rag_chatbot("What happens at the end of the book?")


In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 📚 Book Chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Ask a question about your book...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_reply = rag_chatbot(message)   # use your existing function
        chat_history.append((message, bot_reply))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()
